# Pruebas DataHub Grupo Scanda

## Generación de WindowKey

Esta función ha sido implementada de manera exitosa en el programa principal <i>dhOperations.py</i>

In [1]:
%pip install jellyfish

In [2]:
import jellyfish
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("Muestra1.csv", delimiter=";")
data.head(5)

C:\Users\carav\AppData\Local\Temp\ipykernel_10388\3679848993.py:1: DtypeWarning: Columns (19,31,56,99,100,104,105,106,111,112,113,114,115,116,122,123,125,127,177,178,179,180,181,203,204,207,209,227,279,280,281,301,302,307,308,319,321,333,334,335) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("Muestra1.csv", delimiter=";")


,MasterAccountID,MasterAccountName,Merged,StateCode,StatusCode,AccountId,AccountName,AccountCategoryCode,TerritoryId,CustomerSizeCode,...,PostVentaMCAFullName,PostVentaMCAYomiFullName,ServiceAccountID,ServiceAccountName,LeadID,LeadYomiFullName,LeadFullName,ExternalPartyAccountModifiedByID,ExternalPartyAccountModifiedByFullName,ExternalPartyAccountModifiedByYomiFullName
0,NaN,NaN,0,0,1,A47C6F10-E310-E411-80E1-00155D012F03,Servicios Telefonicos HED,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0,0,1,7A3A1444-E610-E411-80E1-00155D012F03,Maquinas de Coser de Chihuahua,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,0,0,1,7110BA29-E910-E411-80E1-00155D012F03,Vargas Valerio Carlos ANtonio,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,0,0,1,01C5D9A8-EA10-E411-80E1-00155D012F03,Ample Mind S de RL de CV,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,0,0,1,23AB4335-EF10-E411-80E1-00155D012F03,CONTINO S.A. DE C.V.,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data.shape

(54651, 339)

In [5]:
full_obj_data = []

for col in data.columns:
    if data[col].dtype == 'O' and data[col].isna().sum() == 0:
        full_obj_data.append(col)

print(full_obj_data)

['AccountId', 'OwningBusinessUnit', 'CreatedOn', 'CreatedBy', 'ModifiedOn', 'ModifiedBy', 'VersionNumber', 'OpenDeals_Date', 'CustomerAddressId', 'CustomerAddressId12', 'OwnerId', 'OwnerName', 'OwnerYomiName', 'OwningUser', 'CreatedByID', 'CreatedByYomiFullName', 'ModifiedByID35', 'ModifiedByFullName', 'SystemUserID38', 'SystemUserFullName39']


In [23]:
def windowKey(data:pd.DataFrame, cols, methods, substring_length=3, upper=True):
    
    """
    Función encargada de generar una window key para los datos utilizando las
    columnas y métodos especificados por el usuario.
    
    Argumentos:
        data (DataFrame) dataframe que contiene la base de datos.
        cols (lst) lista de columnas a utilizar para generar la window key.
        methods (lst) lista de métodos a utilizar. Para soundex el método es
            'sou' y para substring es 'sub'.
        substring_length (int) número entero que nos indica la longitud que
            tendrá el substring.
        upper (bool) valor booleano que nos indica si el substring será en
            minúscula o mayúscula.
        
    Regresa:
        una lista que contiene las window keys generadas para el dataset en
        cuestión.
        
    """

    func_dict = {i: [cols[i], methods[i]] for i in range(len(cols))}

    windowKey = []

    for i in range(len(func_dict)):
        
        if func_dict[i][1] == "sou":
            soundex = np.vectorize(jellyfish.soundex)
            windowKey.append(soundex(data[func_dict[i][0]].values))
        if func_dict[i][1] == "sub":
            if upper == True:
                substring = np.vectorize(lambda x: x[:substring_length].replace(' ', '').upper() if (len(x) == substring_length) else (x[:substring_length].replace(' ', '').upper() + (' ' * (substring_length - len(x)))))
                windowKey.append(substring(data[func_dict[i][0]].values))
            else:
                substring = np.vectorize(lambda x: x[:substring_length].replace(' ', '') if (len(x) == substring_length) else (x[:substring_length].replace(' ', '') + (' ' * (substring_length - len(x)))))
                windowKey.append(substring(data[func_dict[i][0]].values))

    return list(map(''.join, zip(*windowKey)))

data["WindowKey"] = windowKey(data, ["OwnerName", "OwnerYomiName", "OwningUser", "CreatedByYomiFullName", "ModifiedByFullName"], ["sou", "sub", "sou", "sou", "sub"], 10, True)
data.head(5)

,MasterAccountID,MasterAccountName,Merged,StateCode,StatusCode,AccountId,AccountName,AccountCategoryCode,TerritoryId,CustomerSizeCode,...,PostVentaMCAYomiFullName,ServiceAccountID,ServiceAccountName,LeadID,LeadYomiFullName,LeadFullName,ExternalPartyAccountModifiedByID,ExternalPartyAccountModifiedByFullName,ExternalPartyAccountModifiedByYomiFullName,WindowKey
0,NaN,NaN,0,0,1,A47C6F10-E310-E411-80E1-00155D012F03,Servicios Telefonicos HED,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D523DAYANAGUAD111D523DAYANAGUA
1,NaN,NaN,0,0,1,7A3A1444-E610-E411-80E1-00155D012F03,Maquinas de Coser de Chihuahua,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D523DAYANAGUAD111D523DAYANAGUA
2,NaN,NaN,0,0,1,7110BA29-E910-E411-80E1-00155D012F03,Vargas Valerio Carlos ANtonio,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F142FABIOLASA2111F142FABIOLASA
3,NaN,NaN,0,0,1,01C5D9A8-EA10-E411-80E1-00155D012F03,Ample Mind S de RL de CV,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A536ANDREAMAR9113A536ANDREAMAR
4,NaN,NaN,0,0,1,23AB4335-EF10-E411-80E1-00155D012F03,CONTINO S.A. DE C.V.,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,J222JESUSGARC2111J222PROCRMAD


In [ ]:
data2 = pd.read_table("Muestra2.txt", encoding="ISO-8859-1")

C:\Users\carav\AppData\Local\Temp\ipykernel_21328\3934066765.py:1: DtypeWarning: Columns (9,12,16,18,21,24) have mixed types. Specify dtype option on import or set low_memory=False.
  data2 = pd.read_table("Muestra2.txt", encoding="ISO-8859-1")


In [ ]:
data2.shape

(1301722, 30)

In [ ]:
data2.head(10)

,ï»¿ProspectID,SiteID,ProspectNumber,SourceID,Salutation,LastName,FirstName,StreetAddress,City,County,...,SpouseName,OccupationID,Occupation,HouseholdIncome,PreferredLanguageID,PreferredLanguage,Steward,NationalityID,IncomeID,StateID
0,1111638,155.0,1111638,NaN,NaN,Ruelas Alvarez,Miguel Antonio,28 135,Ciudad Nezahualcoyot,NaN,...,NaN,362.0,Technology,NaN,249.0,Spanish,NaN,479.0,NaN,51.0
1,1111639,155.0,1111639,NaN,NaN,Juarez Lopez,Juan David,Mar De Cortes Edif 3 Dept 501,Cuernavaca,NaN,...,NaN,331.0,Employee,NaN,249.0,Spanish,NaN,479.0,NaN,56.0
2,1111640,102.0,1111640,NaN,NaN,Parize,Danilo,Rua Jose Risstito 463,Sao Paulo,NaN,...,NaN,NaN,NaN,NaN,247.0,Portugues,NaN,459.0,NaN,2654.0
3,1111641,127.0,1111641,NaN,NaN,Lanvier,Forni,Mamieu 238 Rafaera Pcia Santa Fe,Pci Santa Fe,NaN,...,NaN,NaN,NaN,NaN,249.0,Spanish,NaN,454.0,NaN,2073.0
4,1111642,9.0,1111642,NaN,NaN,Rojas MartÃ­nez,Alma,Calle Framasa Garcia Batle Edificio 5b Depto 301,Mexico,NaN,...,NaN,NaN,NaN,NaN,249.0,Spanish,yescuderoaca,479.0,NaN,26.0
5,1111643,0.0,1111643,NaN,Mr.,Miller,Gregory Brent,3530 Diane Place,Greeley,NaN,...,NaN,355.0,Sales Person,0.0,240.0,English,NaN,453.0,501.0,20.0
6,1111644,9.0,1111644,NaN,NaN,Rojas Martinez ...,Alma ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1111645,30.0,1111645,NaN,Sr.,Olvera Osornio,Martin Pedro,Gutierrez Najera Sur 67 Col,Queretaro,NaN,...,NaN,NaN,NaN,NaN,249.0,Spanish,itesta,479.0,NaN,1911.0
8,1111646,127.0,1111646,NaN,NaN,Bes Pena,Jose Luis,Collblanc 155 L Hospitalet De Llobregat,Collblanc,NaN,...,NaN,NaN,NaN,NaN,249.0,Spanish,NaN,454.0,NaN,2073.0
9,1111647,30.0,1111647,NaN,Sr.,Castillo Escalonia,Felix Antonio,Ur Bella Florida Sec 2,Valencia,NaN,...,NaN,NaN,NaN,NaN,249.0,Spanish,NaN,462.0,NaN,422.0


In [ ]:
selected_cols = ["LastName", "FirstName", "City"]

for col in selected_cols:
    data2[col].fillna(data2[col].mode()[0], inplace=True)
    print(data2[col].isna().sum())

0
0
0


In [ ]:
data2["WindowKey"] = windowKey(data=data2, cols=["LastName", "FirstName", "City"], methods=["sub", "sou", "sou"], substring_length=10)

In [ ]:
data2.head(5)

,ï»¿ProspectID,SiteID,ProspectNumber,SourceID,Salutation,LastName,FirstName,StreetAddress,City,County,...,OccupationID,Occupation,HouseholdIncome,PreferredLanguageID,PreferredLanguage,Steward,NationalityID,IncomeID,StateID,WindowKey
0,1111638,155.0,1111638,NaN,NaN,Ruelas Alvarez,Miguel Antonio,28 135,Ciudad Nezahualcoyot,NaN,...,362.0,Technology,NaN,249.0,Spanish,NaN,479.0,NaN,51.0,RUELASALVM245C335
1,1111639,155.0,1111639,NaN,NaN,Juarez Lopez,Juan David,Mar De Cortes Edif 3 Dept 501,Cuernavaca,NaN,...,331.0,Employee,NaN,249.0,Spanish,NaN,479.0,NaN,56.0,JUAREZLOPJ531C651
2,1111640,102.0,1111640,NaN,NaN,Parize,Danilo,Rua Jose Risstito 463,Sao Paulo,NaN,...,NaN,NaN,NaN,247.0,Portugues,NaN,459.0,NaN,2654.0,PARIZE D540S140
3,1111641,127.0,1111641,NaN,NaN,Lanvier,Forni,Mamieu 238 Rafaera Pcia Santa Fe,Pci Santa Fe,NaN,...,NaN,NaN,NaN,249.0,Spanish,NaN,454.0,NaN,2073.0,LANVIER F650P225
4,1111642,9.0,1111642,NaN,NaN,Rojas MartÃ­nez,Alma,Calle Framasa Garcia Batle Edificio 5b Depto 301,Mexico,NaN,...,NaN,NaN,NaN,249.0,Spanish,yescuderoaca,479.0,NaN,26.0,ROJASMARTA450M220


In [ ]:
data2["WindowKey"]

0           RUELASALVM245C335
1           JUAREZLOPJ531C651
2          PARIZE    D540S140
3          LANVIER   F650P225
4           ROJASMARTA450M220
                  ...        
1301717    BAJZ      A641O412
1301718    LUCERO    D265C100
1301719    SABAG     M254C100
1301720     GUTIERREZR560C100
1301721    TIXEIRA   C642E214
Name: WindowKey, Length: 1301722, dtype: object

## Función Sort

In [ ]:
data.head(10)

,MasterAccountID,MasterAccountName,Merged,StateCode,StatusCode,AccountId,AccountName,AccountCategoryCode,TerritoryId,CustomerSizeCode,...,PostVentaMCAYomiFullName,ServiceAccountID,ServiceAccountName,LeadID,LeadYomiFullName,LeadFullName,ExternalPartyAccountModifiedByID,ExternalPartyAccountModifiedByFullName,ExternalPartyAccountModifiedByYomiFullName,WindowKey
0,NaN,NaN,0,0,1,A47C6F10-E310-E411-80E1-00155D012F03,Servicios Telefonicos HED,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D523DAYANAGUAD111D523DAYANAGUA
1,NaN,NaN,0,0,1,7A3A1444-E610-E411-80E1-00155D012F03,Maquinas de Coser de Chihuahua,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D523DAYANAGUAD111D523DAYANAGUA
2,NaN,NaN,0,0,1,7110BA29-E910-E411-80E1-00155D012F03,Vargas Valerio Carlos ANtonio,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F142FABIOLASA2111F142FABIOLASA
3,NaN,NaN,0,0,1,01C5D9A8-EA10-E411-80E1-00155D012F03,Ample Mind S de RL de CV,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A536ANDREAMAR9113A536ANDREAMAR
4,NaN,NaN,0,0,1,23AB4335-EF10-E411-80E1-00155D012F03,CONTINO S.A. DE C.V.,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,J222JESUSGARC2111J222PROCRMAD
5,NaN,NaN,0,0,1,71AD0475-F010-E411-80E1-00155D012F03,Auto Lineas JR.,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,J216JOSEFERNA4111A536JOSEFERNA
6,NaN,NaN,0,0,1,18AA4BB7-F110-E411-80E1-00155D012F03,SYS 21 SA de CV,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F155FABIANANDE131C434LUISFERNA
7,NaN,NaN,0,0,1,EB914DEB-F610-E411-80E1-00155D012F03,Centro Medico Juan Pablo II,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C645CAROLINACD131C645PROCRMAD
8,NaN,NaN,0,0,1,855D0697-0911-E411-80E1-00155D012F03,AYUDA LEGAL PARA LA MUJER,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M655MARIANANC1121M655MARIANANC
9,NaN,NaN,0,0,1,B0A67EFC-1611-E411-80E1-00155D012F03,Hotel Bugambilias,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O564OMARALEJA3133N462ALMAKARIN


In [ ]:
data.sort_values(by="WindowKey", ascending=True, kind='quicksort', inplace=True)
data.index = [x for x in range(data.shape[0])]
data.head(10)

,MasterAccountID,MasterAccountName,Merged,StateCode,StatusCode,AccountId,AccountName,AccountCategoryCode,TerritoryId,CustomerSizeCode,...,PostVentaMCAYomiFullName,ServiceAccountID,ServiceAccountName,LeadID,LeadYomiFullName,LeadFullName,ExternalPartyAccountModifiedByID,ExternalPartyAccountModifiedByFullName,ExternalPartyAccountModifiedByYomiFullName,WindowKey
0,NaN,NaN,0,0,1,37B930FC-4DC2-E611-80CA-005056A86745,rloza test,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A142APLICACION3112A142APLICACION
1,NaN,NaN,0,0,1,F8547087-D0C6-E611-80CA-005056A86745,Sastre,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A142APLICACION3112A142APLICACION
2,NaN,NaN,0,0,1,142AA1A6-95C1-E611-80CA-005056A86745,Isai de Loza,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A142APLICACION3112A142APLICACION
3,NaN,NaN,0,0,1,37A604C8-B5C3-E611-80CA-005056A86745,CABLES AROMANO,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A142APLICACION3112A142APLICACION
4,NaN,NaN,0,0,1,5C634349-95C1-E611-80CA-005056A86745,testing2016,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A142APLICACION3112A142APLICACION
5,NaN,NaN,0,0,1,AEF29ED1-5FC2-E611-80CA-005056A86745,cliente nuevo,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A142APLICACION3112A142FRANCISCO
6,NaN,NaN,0,0,1,D2B486F2-38DC-E611-80CA-005056A86745,Tl del sur,NaN,NaN,1,...,NaN,NaN,NaN,2C99BA9E-78D4-E611-80CA-005056A86745,Miguel mayen,Miguel mayen,NaN,NaN,NaN,A142APLICACION3112E565EMMARIANA
7,NaN,NaN,0,0,1,A39E5530-37DC-E611-80CA-005056A86745,JACK ENGLE DE MEXICO,NaN,NaN,1,...,NaN,NaN,NaN,2D76880D-80D6-E611-80CA-005056A86745,JAIME NUÑEZ,JAIME NUÑEZ,NaN,NaN,NaN,A142APLICACION3112E565EMMARIANA
8,NaN,NaN,0,0,1,5FA3723A-E6F2-E611-80CA-005056A86745,ENLACE DIGITAL,NaN,NaN,1,...,NaN,NaN,NaN,C647DE41-C4EF-E611-80CA-005056A86745,LILIA GONZALEZ,LILIA GONZALEZ,NaN,NaN,NaN,A142APLICACION3112E565EMMARIANA
9,NaN,NaN,0,0,1,558FE279-E525-E711-80CA-005056A86745,Grupo IPS,NaN,NaN,1,...,NaN,NaN,NaN,102E3327-E71E-E711-80CA-005056A86745,idania rios,idania rios,NaN,NaN,NaN,A142APLICACION3112E565EMMARIANA


In [ ]:
def sortWindowKey(data:pd.DataFrame, wk="WindowKey"):
    data.sort_values(by=wk, ascending=True, kind='quicksort', inplace=True)
    data.index = [x for x in range(data.shape[0])]
    return True

In [ ]:
data2.head(10)

,ï»¿ProspectID,SiteID,ProspectNumber,SourceID,Salutation,LastName,FirstName,StreetAddress,City,County,...,OccupationID,Occupation,HouseholdIncome,PreferredLanguageID,PreferredLanguage,Steward,NationalityID,IncomeID,StateID,WindowKey
0,1286049,0.0,1286049,1643.0,NaN,,Angel Gonzalez,",",Mexico,United States,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,A524M220
1,1304776,6.0,1304776,1643.0,NaN,,antonio puente del pozo,"Reyna Maria Antonieta COL. LOS REYES ,",San Luis Potosi,Mexico,...,12.0,Comerciante,0.0,0.0,NaN,NaN,NaN,NaN,NaN,A535S542
2,1109583,3.0,1109583,1643.0,NaN,,Amador,,Mexico,NaN,...,366.0,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,40.0,A536M220
3,1285877,3.0,1285877,1643.0,NaN,,Cesar Valle,"Caminata Douro 11070 DOURO ,",san diego,United States,...,9.0,Carpenter,0.0,0.0,NaN,NaN,NaN,NaN,NaN,C261S532
4,1285432,3.0,1285432,1643.0,NaN,,JHOANA,"GARDENIA #915 COL. REPUBLICA MEXICANA ,",CULIACAN,Mexico,...,34.0,Nurse,0.0,0.0,NaN,NaN,NaN,NaN,NaN,J500C425
5,1307653,6.0,1307653,1643.0,NaN,,miguel salvador cerna lara,Bernal Diaz del Castillo #27 COL. CIRCUITO HIS...,Satelite,Mexico,...,7.0,Banker,0.0,0.0,NaN,NaN,NaN,NaN,NaN,M242S343
6,1276946,3.0,1276946,1643.0,NaN,,MARCO ANTONIO,"MCLAUGHIN AU #3 2223 COL. ,",SAN JOSE,United States,...,21.0,Employee,0.0,0.0,NaN,NaN,NaN,NaN,NaN,M625S522
7,1270822,5.0,1270822,271.0,NaN,,pedro ornelo,"n COL.N , carolina",carolina,Puerto Rico,...,29.0,Lawyer,0.0,0.0,NaN,NaN,NaN,NaN,NaN,P366C645
8,1131019,1.0,1131019,NaN,NaN,...,...,NaN,Mexico,NaN,...,NaN,NaN,NaN,NaN,NaN,GLUNAC,NaN,NaN,NaN,000M220
9,1115259,401.0,1115259,NaN,NaN,...,...,NaN,Mexico,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000M220


In [ ]:
print(sortWindowKey(data2))
data2.head(10)

True


,ï»¿ProspectID,SiteID,ProspectNumber,SourceID,Salutation,LastName,FirstName,StreetAddress,City,County,...,OccupationID,Occupation,HouseholdIncome,PreferredLanguageID,PreferredLanguage,Steward,NationalityID,IncomeID,StateID,WindowKey
0,1286049,0.0,1286049,1643.0,NaN,,Angel Gonzalez,",",Mexico,United States,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,A524M220
1,1304776,6.0,1304776,1643.0,NaN,,antonio puente del pozo,"Reyna Maria Antonieta COL. LOS REYES ,",San Luis Potosi,Mexico,...,12.0,Comerciante,0.0,0.0,NaN,NaN,NaN,NaN,NaN,A535S542
2,1109583,3.0,1109583,1643.0,NaN,,Amador,,Mexico,NaN,...,366.0,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,40.0,A536M220
3,1285877,3.0,1285877,1643.0,NaN,,Cesar Valle,"Caminata Douro 11070 DOURO ,",san diego,United States,...,9.0,Carpenter,0.0,0.0,NaN,NaN,NaN,NaN,NaN,C261S532
4,1285432,3.0,1285432,1643.0,NaN,,JHOANA,"GARDENIA #915 COL. REPUBLICA MEXICANA ,",CULIACAN,Mexico,...,34.0,Nurse,0.0,0.0,NaN,NaN,NaN,NaN,NaN,J500C425
5,1307653,6.0,1307653,1643.0,NaN,,miguel salvador cerna lara,Bernal Diaz del Castillo #27 COL. CIRCUITO HIS...,Satelite,Mexico,...,7.0,Banker,0.0,0.0,NaN,NaN,NaN,NaN,NaN,M242S343
6,1276946,3.0,1276946,1643.0,NaN,,MARCO ANTONIO,"MCLAUGHIN AU #3 2223 COL. ,",SAN JOSE,United States,...,21.0,Employee,0.0,0.0,NaN,NaN,NaN,NaN,NaN,M625S522
7,1270822,5.0,1270822,271.0,NaN,,pedro ornelo,"n COL.N , carolina",carolina,Puerto Rico,...,29.0,Lawyer,0.0,0.0,NaN,NaN,NaN,NaN,NaN,P366C645
8,1130449,401.0,1130449,NaN,NaN,...,...,NaN,Mexico,NaN,...,NaN,NaN,NaN,NaN,NaN,EVALLE,NaN,NaN,NaN,000M220
9,1115860,401.0,1115860,NaN,NaN,...,...,NaN,Mexico,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000M220
